In [ ]:
%pip install datasets
%pip install huggingface_hub
%pip install tokenizers
%pip install transformers
%pip install tensorboard

from google.colab import userdata

# Get the Hugging Face token from userdata (if set)
hf_token = userdata.get('HF_TOKEN')

# Use the token to log in
from huggingface_hub import login
login(hf_token)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
Dataset = load_dataset('chillies/IELTS_evaluations')

train.csv:   0%|          | 0.00/48.2M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9912 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/495 [00:00<?, ? examples/s]

In [ ]:
# Access the train and test splits
train_dataset = Dataset['train']
test_dataset = Dataset['test']

In [ ]:
import pandas as pd

# Convert train_data and test_data to pandas DataFrame
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)

In [ ]:
# prompt: check for the null values in both the dataset of the prompt, essay and evaluation columns and if null values drop the row

# Check for null values in 'prompt', 'essay', and 'evaluation' columns of train_df
train_df = train_df.dropna(subset=['prompt', 'essay', 'evaluation', 'band'])

# Check for null values in 'prompt', 'essay', and 'evaluation' columns of test_df
test_df = test_df.dropna(subset=['prompt', 'essay', 'evaluation', 'band'])

In [ ]:
import pandas as pd
import re

# Function to clean text
def clean_text(text):
    # Remove unwanted special characters but keep alphabets, digits, :, (, )
    return re.sub(r'[^\w\s:().\-0-9a-zA-Z]', '', text)

# Apply cleaning to specific columns in train_df and test_df
columns_to_clean = ['evaluation', 'prompt', 'essay']

for col in columns_to_clean:
    train_df[col] = train_df[col].apply(clean_text)
    test_df[col] = test_df[col].apply(clean_text)

In [ ]:
# Combine the columns with <sep> and <eos>
train_df['combined'] = train_df['prompt'] + '\n' + train_df['essay'] + ' <SEP> ' + train_df['evaluation'] + ' <EOS> '
test_df['combined'] = test_df['prompt'] + '\n' + test_df['essay'] + ' <SEP> ' + test_df['evaluation'] + ' <EOS> '

# Save to input.txt
with open('input.txt', 'w') as file:
    for item in train_df['combined']:
        file.write("%s\n" % item)

with open('test.txt', 'w') as file:
    for item in test_df['combined']:
        file.write("%s\n" % item)

print("File saved as input.txt")
print("File saved as test.txt")

File saved as input.txt
File saved as test.txt


In [ ]:
with open('/content/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

with open('/content/test.txt', 'r', encoding='utf-8') as f:
    test_text = f.read()

In [ ]:
from tokenizers import ByteLevelBPETokenizer
from torch.nn.utils.rnn import pad_sequence
import torch
from collections import Counter

# Special tokens
special_tokens = ["<OOV>", "<PAD>", "<SEP>", "<EOS>"]

# Initialize and train tokenizer
tokenizer = ByteLevelBPETokenizer()

# Combine training text from input and evaluation columns
train_text = text
# print(train_text)

# Count characters and words
char_counter = Counter("".join(train_text))
print(f"Number of unique characters: {len(char_counter)}")

word_counter = Counter(" ".join(train_text).split())
vocab_size = len(word_counter)
print(f"Number of unique words: {vocab_size}")

# Train tokenizer
tokenizer.train_from_iterator(
    train_text,
    vocab_size=vocab_size,
    min_frequency=2,
    special_tokens=special_tokens
)

# Define special token IDs
sep_token_id = tokenizer.token_to_id("<SEP>")
eos_token_id = tokenizer.token_to_id("<EOS>")
pad_token_id = tokenizer.token_to_id("<PAD>")
oov_token_id = tokenizer.token_to_id("<OOV>")

Number of unique characters: 282
Number of unique words: 275


In [ ]:
# Encode the training text and test text
train_data = torch.tensor(tokenizer.encode(text).ids, dtype=torch.long)
val_data = torch.tensor(tokenizer.encode(test_text).ids, dtype=torch.long)

# Print the first 100 tokens of each
print("Train data tokens of training data:", train_data)
print("Val data tokens of validation data:", val_data)

Train data tokens of training data: tensor([ 60,  82,  88,  ...,   3, 224, 202])
Val data tokens of validation data: tensor([ 44,  81, 224,  ...,   3, 224, 202])


In [ ]:
train_data_100 = train_data[:100]
val_data_100 = val_data[:100]

print(tokenizer.decode(val_data_100.tolist()))
print(tokenizer.decode(train_data_100.tolist()))

In many countries more and more young people are leaving school and unable to find job after graduat
Young people who commit crimes should be treated in the same way as adults who commit crimes.

To wh


In [ ]:
# Get the vocabulary
vocab = tokenizer.get_vocab()

# Calculate the maximum vocabulary length
max_vocab_length = len(vocab)
print(f"Maximum vocabulary length: {max_vocab_length}")

Maximum vocabulary length: 275


In [ ]:
import torch
import json
import os
import torch.nn as nn
from torch.nn import functional as F
from huggingface_hub import HfApi, Repository
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TextStreamer, AutoModel, AutoTokenizer

# hyperparameters
vocab_size = max_vocab_length
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ------------

torch.manual_seed(1337)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            # print(X, '\n estimate_loss:', Y)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # print(x.shape)
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        idx = idx.clamp(0, vocab_size - 1)

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))

        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)

        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            # Flatten the logits and targets to match shapes for cross entropy
            logits = logits.view(B * T, vocab_size)  # Flatten logits
            targets = targets.view(B * T)  # Flatten targets to a 1D tensor
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

            if any(token.item() == eos_token_id for token in idx_next):
                break

        return idx

    def generate_with_streaming(self, idx, max_new_tokens):
            streamer = TextStreamer(tokenizer)  # Create a TextStreamer instance

            for _ in range(max_new_tokens):
                idx_cond = idx[:, -block_size:]
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
                idx = torch.cat((idx, idx_next), dim=1)

                # Stream the token
                # Changed line: Pass idx_next directly instead of idx_next[0].item()
                streamer.put(idx_next[0])

                if eos_token_id is not None and idx_next.item() == eos_token_id:
                    break

            streamer.end()  # Ensure proper termination
            return tokenizer.decode(idx.tolist()[0])  # Return the final generated text

# Define the configuration class for Hugging Face
class TransformerConfig(PretrainedConfig):
    def __init__(self, vocab_size=None, n_embd=None, n_head=None, n_layer=None, dropout=None, block_size=None, **kwargs): # Added default values for arguments
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.n_embd = n_embd
        self.n_head = n_head
        self.n_layer = n_layer
        self.dropout = dropout
        self.block_size = block_size

model = TransformerModel()
model = model.to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

repo_id = "R0b000/IELTS_DECODER_ONLY"

# Change checkpoint_path to include a filename
checkpoint_path = os.path.join("./checkpoint", "checkpoint_{}.pt".format(iter))

# Save model manually
model_path = "./checkpoint"

# Create the directory if it doesn't exist
os.makedirs(model_path, exist_ok=True)

train_losses = []
val_losses = []

# Training loop
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # Save checkpoint
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'iteration': iter,
            'losses': losses
        }
        torch.save(checkpoint, checkpoint_path)
        print(f"Checkpoint saved at iteration {iter}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_losses.append(losses['train'].item())
    val_losses.append(losses['val'].item())

# Save the model state dictionary (for retraining)
torch.save(model.state_dict(), f"{model_path}/pytorch_model.bin")

# Save optimizer state
torch.save(optimizer.state_dict(), f"{model_path}/optimizer.pth")

# Save the tokenizer
tokenizer.save_model(model_path)
tokenizer.save("tokenizer.json")

# Initialize the configuration with your model's parameters
config = TransformerConfig(
    vocab_size=vocab_size,  # Pass the vocab_size (e.g., the size of your tokenizer's vocabulary)
    n_embd=n_embd,          # Pass the embedding size
    n_head=n_head,          # Number of attention heads
    n_layer=n_layer,        # Number of layers
    dropout=dropout,        # Dropout rate
    block_size=block_size   # Block size (length of the input sequences)
)

# Save the configuration to the model path
config.save_pretrained(model_path)

# Save training loss history
loss_history = [{"iteration": i, "train_loss": loss, "val_loss": val_loss} for i, (loss, val_loss) in enumerate(zip(train_losses, val_losses))]
with open(f"{model_path}/loss_history.json", "w") as f:
    json.dump(loss_history, f)

# Save training metadata (latest checkpoint info)
training_metadata = {
    "iteration": len(loss_history),
    "latest_train_loss": loss_history[-1]["train_loss"] if loss_history else None,
    "latest_val_loss": loss_history[-1]["val_loss"] if loss_history else None,
    "hyperparameters": config.to_dict()  # Assuming config contains model hyperparameters
}
with open(f"{model_path}/training_metadata.json", "w") as f:
    json.dump(training_metadata, f)

print("Model, tokenizer, config, and loss history saved successfully!")

10.950419 M parameters
step 0: train loss 5.8640, val loss 5.8635
Checkpoint saved at iteration 0
step 500: train loss 1.2152, val loss 1.2246
Checkpoint saved at iteration 500
step 1000: train loss 0.8345, val loss 0.8472
Checkpoint saved at iteration 1000
step 1500: train loss 0.7608, val loss 0.7620
Checkpoint saved at iteration 1500
step 2000: train loss 0.7163, val loss 0.7243
Checkpoint saved at iteration 2000
step 2500: train loss 0.6771, val loss 0.6873
Checkpoint saved at iteration 2500
step 3000: train loss 0.6701, val loss 0.6739
Checkpoint saved at iteration 3000
step 3500: train loss 0.6451, val loss 0.6486
Checkpoint saved at iteration 3500
step 4000: train loss 0.6303, val loss 0.6394
Checkpoint saved at iteration 4000
step 4500: train loss 0.6227, val loss 0.6256
Checkpoint saved at iteration 4500
step 4999: train loss 0.6091, val loss 0.6134
Checkpoint saved at iteration 4999
Model, tokenizer, config, and loss history saved successfully!


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_folder(
    folder_path= './checkpoint',  # Path to your model folder
    path_in_repo="",  # Optional, use this if you want to specify a folder in the repo
    repo_id=repo_id  # The repo you created on Hugging Face
)

print("Model and tokenizer successfully uploaded to Hugging Face.")

checkpoint_<built-in function iter>.pt:   0%|          | 0.00/141M [00:00<?, ?B/s]

full_model.pt:   0%|          | 0.00/53.4M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/53.3M [00:00<?, ?B/s]

optimizer.pth:   0%|          | 0.00/87.7M [00:00<?, ?B/s]

Model and tokenizer successfully uploaded to Hugging Face.


### Example 1

In [ ]:
# Example input
input_text = """
Prompt: Some people believe that the best way to reduce crime is to give longer prison sentences. Others believe there are better alternatives. Discuss both views and give your opinion.

Essay:
Crime is a major issue in many societies today, and it is widely debated how best to tackle it. While some argue that longer prison sentences are the most effective deterrent, others suggest that there are alternative methods of reducing crime. In this essay, I will examine both viewpoints before offering my own opinion.

On one hand, proponents of longer prison sentences argue that such measures ensure that criminals are removed from society for longer periods, reducing the likelihood of reoffending. In addition, the fear of long prison terms may discourage potential offenders from engaging in criminal activities. They believe that the justice system must be firm to maintain order and safety.

On the other hand, opponents of this approach argue that incarceration alone does not address the root causes of crime, such as poverty, lack of education, and social inequality. They propose alternatives like rehabilitation programs, education, and community-based initiatives that focus on helping offenders reintegrate into society. These methods aim to reduce recidivism by tackling the social factors that contribute to criminal behavior.

In my opinion, while longer prison sentences may be necessary for certain severe crimes, they should not be the only solution. It is equally important to invest in rehabilitation and education programs to address the causes of crime and reduce the likelihood of reoffending. A balanced approach, combining punishment with rehabilitation, would likely be the most effective way to reduce crime in the long term.
"""

In [ ]:
context = tokenizer.encode(input_text).ids  # ensure the special tokens are present
context_tensor = torch.tensor(context, dtype=torch.long, device=device).unsqueeze(0)

In [ ]:
print(tokenizer.decode(context))


Prompt: Some people believe that the best way to reduce crime is to give longer prison sentences. Others believe there are better alternatives. Discuss both views and give your opinion.

Essay:
Crime is a major issue in many societies today, and it is widely debated how best to tackle it. While some argue that longer prison sentences are the most effective deterrent, others suggest that there are alternative methods of reducing crime. In this essay, I will examine both viewpoints before offering my own opinion.

On one hand, proponents of longer prison sentences argue that such measures ensure that criminals are removed from society for longer periods, reducing the likelihood of reoffending. In addition, the fear of long prison terms may discourage potential offenders from engaging in criminal activities. They believe that the justice system must be firm to maintain order and safety.

On the other hand, opponents of this approach argue that incarceration alone does not address the roo

In [ ]:
generated_output = model.generate(context_tensor, max_new_tokens=2000)
print(tokenizer.decode(generated_output[0].tolist()))


Prompt: Some people believe that the best way to reduce crime is to give longer prison sentences. Others believe there are better alternatives. Discuss both views and give your opinion.

Essay:
Crime is a major issue in many societies today, and it is widely debated how best to tackle it. While some argue that longer prison sentences are the most effective deterrent, others suggest that there are alternative methods of reducing crime. In this essay, I will examine both viewpoints before offering my own opinion.

On one hand, proponents of longer prison sentences argue that such measures ensure that criminals are removed from society for longer periods, reducing the likelihood of reoffending. In addition, the fear of long prison terms may discourage potential offenders from engaging in criminal activities. They believe that the justice system must be firm to maintain order and safety.

On the other hand, opponents of this approach argue that incarceration alone does not address the roo

In [ ]:
generated_output = model.generate_with_streaming(context_tensor, max_new_tokens=2000)


Therefore a prone animal ph Universities happening in AK a goal doctings issue the daily.   Task Achievement:

The candidate has adequately addressed the given task by prelevant ideas of latter and support for their arguments with specific examples or evidence. However the lack of distance and fail to meet diseases across the history of a high job straight and fluctuate ethoration.

Logical flow of ideas are not weakned throughout controlled some aspects and professional inerquility and way because a tention of resource various jobs. As cause of the best was famous of the history of their houses and nored as a strong work.  Task Achievement:
- The candidate has adequately addressed the task by discussing the requirements of the benefits and discoverse caused on who takes their opinion in while others think that they should require a homes to improve overall expense or ecorrecving them.
- Suggested Band Score (Task Achievement): 6

 Coherence and Cohesion:
- The essay is well-organized

### Example 2

In [ ]:
# Example input
input_text_1 = """
Prompt: Some people think that children should be given less homework. Others believe that homework is essential for a child's development. Discuss both views and give your opinion.

Essay:
The issue of whether children should be assigned less homework has sparked a great deal of debate. While some believe that reducing homework would benefit children’s well-being, others argue that homework is vital for academic success and personal growth. In this essay, I will explore both sides of the argument and offer my own perspective.

Supporters of less homework argue that children already have a busy schedule with school, extracurricular activities, and family commitments. Excessive homework can lead to stress, burnout, and a lack of free time, which is crucial for a child’s overall development. Furthermore, it is suggested that children need time to relax and engage in other activities, such as sports or hobbies, to develop a well-rounded personality.

On the contrary, those who advocate for homework argue that it plays a critical role in reinforcing what is taught in the classroom. Homework allows children to practice and apply what they have learned, leading to better retention and understanding. It also helps children develop important skills such as time management, discipline, and responsibility, which are essential for future success.

In my opinion, while homework is necessary for academic development, there should be a balance to ensure that children are not overwhelmed. Assigning reasonable amounts of homework that reinforce key concepts and encourage independent learning, without encroaching on their personal time, would be a better approach.
"""

In [ ]:
context_1 = tokenizer.encode(input_text_1).ids  # ensure the special tokens are present
context_tensor_1 = torch.tensor(context_1, dtype=torch.long, device=device).unsqueeze(0)

In [ ]:
generated_output_1 = model.generate(context_tensor_1, max_new_tokens=2000)
print(tokenizer.decode(generated_output_1[0].tolist()))


Prompt: Some people think that children should be given less homework. Others believe that homework is essential for a child's development. Discuss both views and give your opinion.

Essay:
The issue of whether children should be assigned less homework has sparked a great deal of debate. While some believe that reducing homework would benefit children’s well-being, others argue that homework is vital for academic success and personal growth. In this essay, I will explore both sides of the argument and offer my own perspective.

Supporters of less homework argue that children already have a busy schedule with school, extracurricular activities, and family commitments. Excessive homework can lead to stress, burnout, and a lack of free time, which is crucial for a child’s overall development. Furthermore, it is suggested that children need time to relax and engage in other activities, such as sports or hobbies, to develop a well-rounded personality.

On the contrary, those who advocate f

In [ ]:
generated_output = model.generate_with_streaming(context_tensor_1, max_new_tokens=2000)


Overall Band Score: 4.5

Lexical Resource (Vocabulary):
The candidate demonstrates a limited range of writing writing skills. The use of vocabulary is limited with overall temporary flow of the essay. The candidate language used improving their grammatical accuracy.

Suggested Overall Band Score: 6.5

 Feedback and Additional Comments:
Strengths:
- The essay addresses the given and prompt.
- The candidate provides a well-organized and provides relevant examples.
- The candidate has demonstrated a logical flow of ideas and used some appropriate language throughout their grammar and vocabulary.
- Practice writing grammar and coherently the ideas without a few expercise or language and wide prompt.
- The candidate could use a dictionary or thesaurus to find more specific examples and evidence 


### Example 3

In [ ]:
# Example input
input_text_2 = """
Prompt: Some people believe that climate change is the responsibility of individuals, while others think it is the government's responsibility. Discuss both views and give your opinion.

Essay:
Climate change is one of the most pressing issues of our time, and there is an ongoing debate about who should take responsibility for addressing it. While some argue that individuals must take responsibility for their own actions to combat climate change, others believe that the government should take the lead in tackling this global challenge. In this essay, I will examine both perspectives before offering my own opinion.

On one hand, those who argue that individuals should be responsible for fighting climate change contend that personal actions, such as reducing energy consumption, recycling, and using eco-friendly products, can have a significant impact. They believe that if everyone made small changes in their daily lives, the collective effect could contribute to a reduction in carbon emissions and the overall environmental footprint. Moreover, individual awareness and education can encourage others to adopt more sustainable lifestyles.

On the other hand, opponents of this view argue that climate change is a global issue that requires coordinated government action. They believe that governments have the power to enact policies that can regulate emissions, promote renewable energy, and invest in sustainable infrastructure. For example, governments can introduce laws to limit pollution, provide incentives for green technology, and fund research into alternative energy sources. Therefore, the responsibility for combating climate change should fall on policymakers who have the authority and resources to make systemic changes.

In my opinion, while individuals have a role to play in reducing their environmental impact, the government should take the lead in tackling climate change. Governments have the necessary tools, resources, and influence to implement large-scale changes that can make a significant difference. A joint effort between individuals and governments would be the most effective way to combat this global challenge.

"""

In [ ]:
context_2 = tokenizer.encode(input_text_2).ids  # ensure the special tokens are present
context_tensor_2 = torch.tensor(context_2, dtype=torch.long, device=device).unsqueeze(0)

In [ ]:
generated_output_2 = model.generate(context_tensor_2, max_new_tokens=2000)
print(tokenizer.decode(generated_output_2[0].tolist()))


Prompt: Some people believe that climate change is the responsibility of individuals, while others think it is the government's responsibility. Discuss both views and give your opinion.

Essay:
Climate change is one of the most pressing issues of our time, and there is an ongoing debate about who should take responsibility for addressing it. While some argue that individuals must take responsibility for their own actions to combat climate change, others believe that the government should take the lead in tackling this global challenge. In this essay, I will examine both perspectives before offering my own opinion.

On one hand, those who argue that individuals should be responsible for fighting climate change contend that personal actions, such as reducing energy consumption, recycling, and using eco-friendly products, can have a significant impact. They believe that if everyone made small changes in their daily lives, the collective effect could contribute to a reduction in carbon em

In [ ]:
generated_output = model.generate_with_streaming(context_tensor_2, max_new_tokens=2000)

To begin with the some price of for a childs an encidence the centers alid retirement information more people are tend to descrive the housing should other jobs . For example in the housing relevance payment at even wre high least the lot of a listening for recycling sector in every work people is welling faceful to apply with their time profession. Without financial happening the commonsequences should be improved at to this effort officiency reditulion is an intension) for youngster a lot of of timp and achievalues carefully are paparents in them his only during paintier studies although this is illustritely. This essay also has a better school for this contentrest that wh is difficultied as humans they should forting for research his goods behind to the winterations no teachers their own courses which did no make them. It is a hours rapidly thing their resement kills and it is unsile belier to creation and make mistake themmatically few.

On the other hand I think this days evidence

### Example 4

In [ ]:
# Example input
input_text_3 = """
Prompt: Some people think that studying abroad is essential for students, while others believe that staying in their home country is a better option. Discuss both views and give your opinion.

Essay:
The decision of whether to study abroad or remain in one’s home country is a topic of much debate. Some people argue that studying abroad is essential for a student’s personal and academic growth, while others believe that staying in one’s home country is the better choice. This essay will examine both perspectives before offering my own opinion.

On one hand, those who advocate for studying abroad highlight the benefits of experiencing a new culture, broadening one’s horizons, and learning in a different academic environment. Exposure to diverse teaching methods and a global network of peers can provide students with unique opportunities for growth. Furthermore, studying abroad can help students develop independence, adaptability, and language skills, which are invaluable in today’s globalized job market.

On the other hand, those who believe that staying in one’s home country is preferable argue that students can receive a high-quality education without the added stress and financial burden of studying abroad. Staying at home allows students to remain close to family and friends, which can provide emotional support and stability. Moreover, in many countries, there are reputable universities that offer excellent education and research opportunities, making it unnecessary to leave the country.

In my opinion, studying abroad can offer valuable experiences that contribute to personal and academic growth. However, I also believe that it is important for students to weigh the financial costs, potential cultural challenges, and educational quality when making this decision. Ultimately, the best choice depends on individual circumstances, but studying abroad can offer distinct advantages for those who can afford the experience.
"""

In [ ]:
context_3 = tokenizer.encode(input_text_3).ids  # ensure the special tokens are present
context_tensor_3 = torch.tensor(context_3, dtype=torch.long, device=device).unsqueeze(0)

In [ ]:
generated_output_3 = model.generate(context_tensor_3, max_new_tokens=2000)
print(tokenizer.decode(generated_output_3[0].tolist()))


Prompt: Some people think that studying abroad is essential for students, while others believe that staying in their home country is a better option. Discuss both views and give your opinion.

Essay:
The decision of whether to study abroad or remain in one’s home country is a topic of much debate. Some people argue that studying abroad is essential for a student’s personal and academic growth, while others believe that staying in one’s home country is the better choice. This essay will examine both perspectives before offering my own opinion.

On one hand, those who advocate for studying abroad highlight the benefits of experiencing a new culture, broadening one’s horizons, and learning in a different academic environment. Exposure to diverse teaching methods and a global network of peers can provide students with unique opportunities for growth. Furthermore, studying abroad can help students develop independence, adaptability, and language skills, which are invaluable in today’s glob

In [ ]:
generated_output = model.generate_with_streaming(context_tensor_3, max_new_tokens=2000)

Theselving is brixes a cost of think at doing the less a standard efficient in duty school families they etc.   Task Achievement:
- The candidate has addressed the task by by discussing both views on the issues and providing their own opinion.
- The ideas are presented clearly and coherence but some it could be prevaled to enhanced coherence.
- The candidate has covered all parts of the task. The addressed the prompt by discussing both viewpoints of contemporary interrafe government in coherent mindid.
- Suggested Band Score (Task Achievement): 7.5

 Coherence and Cohesion:
- The essay is well-organized with a logist flow of information.
- The use of effective linking devices is effective.
- Suggested Band Score (Coherence and Cohesion): 7

 Lexical Resource (Vocabulary):
- The essay demonstrates a good range of vocabulary using adepth and sophisticated words.
- However there are a few instances of inaccurate word usage such as overused instead of such checking (prossunding new produci

### Example 5

In [ ]:
# Example input
input_text_4 = """
Prompt: Some people believe that technology has made communication between people easier, while others think it has created more barriers between people. Discuss both views and give your opinion.

Essay:
Advancements in technology have revolutionized communication in many ways, but they have also sparked debates about whether they have brought people closer together or created more barriers. Some believe that technology has made communication easier, while others argue that it has led to greater isolation. In this essay, I will discuss both views and offer my own opinion.

On one hand, technology has undoubtedly made communication easier and more accessible. With the advent of smartphones, social media, and instant messaging apps, people can now stay in touch with family, friends, and colleagues regardless of geographical distance. This has allowed for more frequent and convenient interactions, and has facilitated the exchange of ideas and information across borders. Additionally, video calls and virtual meetings have enabled people to collaborate and communicate professionally without the need for physical presence.

On the other hand, some argue that technology has created new barriers between people. Despite the increased accessibility of communication tools, face-to-face interaction has become less common, leading to a decline in meaningful, personal relationships. People may be more likely to engage with others online than in person, which can lead to feelings of loneliness and social isolation. Furthermore, constant reliance on technology can create distractions, leading to a lack of attention and empathy in conversations.

In my opinion, while technology has made communication more convenient, it is important to find a balance between online and offline interactions. Technology should be used to enhance relationships, not replace them. In this way, people can enjoy the benefits of technological communication while maintaining strong, personal connections with those around them.
"""

In [ ]:
context_4 = tokenizer.encode(input_text_4).ids  # ensure the special tokens are present
context_tensor_4 = torch.tensor(context_4, dtype=torch.long, device=device).unsqueeze(0)

In [ ]:
generated_output_4 = model.generate(context_tensor_4, max_new_tokens=2000)
print(tokenizer.decode(generated_output_4[0].tolist()))


Prompt: Some people believe that technology has made communication between people easier, while others think it has created more barriers between people. Discuss both views and give your opinion.

Essay:
Advancements in technology have revolutionized communication in many ways, but they have also sparked debates about whether they have brought people closer together or created more barriers. Some believe that technology has made communication easier, while others argue that it has led to greater isolation. In this essay, I will discuss both views and offer my own opinion.

On one hand, technology has undoubtedly made communication easier and more accessible. With the advent of smartphones, social media, and instant messaging apps, people can now stay in touch with family, friends, and colleagues regardless of geographical distance. This has allowed for more frequent and convenient interactions, and has facilitated the exchange of ideas and information across borders. Additionally, vid

In [ ]:
generated_output = model.generate_with_streaming(context_tensor_4, max_new_tokens=2000)


Few students arguments will be synonymously during their design to other modremic two increase  games and people want to decrease infrastructures can teach affective their owner.   Task Achievement:
- The candidate has addressed the given task by providing a clear stance and supports for their itain their opinion.
- While there are a few insight of its the number of places lowth things it has become important an increase of traffic and support it will continuied.
- The candidate needs from the reliable environmental Googlament national needs aterve should be taken the most activities can be have some and misjointed.
- Therefore it is uggested in Grammar especial and gas addemntization of the essay lacks a clear solution to enhance ahibitant role of environment and local scientisties on the last matural resource in social media pointed in.
- Suggested Band Score (Grammatical Range and Accuracy): 6.0

 Overall Band Score:
- Considering the overall performance across all criteria the ove